# PROCESSAMENTO DE IMAGEM - PEDUNCULO

In [1]:
def preveArea(xt, yt, xb, yb, largura, altura):
    
    #Regiao de interesse

    Lmax = abs(xt - xb)
    Hmax = abs(yt - yb)

    RoiL = largura * Lmax
    RoiH = altura  * Hmax

    #Esses coordenadas são as que vão delimitar a area onde a imagem vai ser cortada

    centro = (xb + xt) / 2

    centroCaixa = RoiL / 2

    x1 = centro - centroCaixa
    x2 = centro + centroCaixa

    y1 = abs(yt - 50)
    y2 = abs(RoiH - y1)

    x1 = int(x1)
    y1 = int(y1)
    x2 = int(x2)
    y2 = int(y2)
    
    return x1, y1, x2, y2  

In [2]:
def segmentacaoHSV(imgCorte):
    
    baixo = np.array([2, 0, 0])
    alto =  np.array([30, 255, 255])

    img_hsv = cv2.cvtColor(imgCorte, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(img_hsv, baixo, alto)

    img_seg = cv2.bitwise_and(imgCorte, imgCorte, mask = mask)
    
    return img_seg

In [35]:
def maior_valor_H(i, img, distancia):
    
    j = 0
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    largura = img.shape[1]
    altura  = img.shape[0]
    
    centroX = round(largura / 2)
    centroY = round(altura / 2)
    
    todas_coordenadas  =  [] #Guarda todas as coordenadas (eixo X e Y)
    todas_coordenadasX =  [] #Guarda as coordenadas do eixo X vindas dos cantos encontrados
    todas_coordenadasY =  [] #Guarda as coordenadas do eixo Y vindas dos cantos encontrados
        
    for x in range(0, largura):

        for y in range(0, altura):

            h, s, v = img[y, x]

            if(h == 15):

                todas_coordenadas.append((x,y))
                todas_coordenadasX.append(int(x))
                todas_coordenadasY.append(int(y))

                cv2.circle(img, (x, y), 1, (255, 0, 255), -1) #purple

                j = j + 1

            #print((y, x), h,s,v)
            
    tam = len(todas_coordenadas)
    
    if(tam >= 1):
        
        return todas_coordenadas,todas_coordenadasX, todas_coordenadasY, img
    
    else:
        
        todas_coordenadas, todas_coordenadasX, todas_coordenadasY = [(centroX, centroY + (centroY * distancia))], [(centroX)], [(centroY * distancia)]
        
        return todas_coordenadas,todas_coordenadasX, todas_coordenadasY, img

    cv2.imwrite("Teste_H/(" + str(i) + ").png", img)

    plt.imshow(img)
    plt.show

In [4]:
#Funções

#Calcula o peso das coordenadas

def calculaPesoY(a):
    
    calPeso = (a[0] * a[0]) + (a[1] * a[1])
    
    calPeso = np.sqrt(calPeso)
    
    calPeso = int(calPeso)
    
    return calPeso

def calculaPesoX(a, centro):
    
    if(a[0] > centro):
        
        dif = (centro - a[0])
        
        soma = a[0] + dif
        
    else:
        
        soma = a[0]
        
    calPeso = (soma * soma) + (a[1] * a[1])
    
    calPeso = np.sqrt(calPeso)
    
    calPeso = round(calPeso)
    
    return calPeso


#Faz a combinação entre a coordenadas e os pesos

def combinacao(peso: list, todas_coordenadasN: list) -> list:
    
    assert len(peso) == len(todas_coordenadasN)
 
    n = len(peso)
    
    combi = []
 
    for i in range(n):
        
        combi.append((peso[i], todas_coordenadasN[i]))
         
    return combi

#Calcula a media dos pontos

def calculaMedia(m):
    
    calMedia = (m[0] *  m[1])
    
    calMedia = int(calMedia)
    
    return calMedia

In [5]:
def encontrarPontos(img, todasCoordenadas, coordenadasX, coordenadasY):
    
    pesoY =   []
    pesoX =   []
    mediaX =  []
    mediaY =  []
    
    centro =  img.shape[0] / 2
    centro = int(centro)
     
    #Calcula o peso de Y
    for i in todasCoordenadas:

        aux = calculaPesoY(i)

        pesoY.append(aux)

    #Calcula o peso de X
    for i in todasCoordenadas:

        aux = calculaPesoX(i, centro)

        pesoX.append(aux)

    #Somatorio do peso Y
    somatorioPesoY = sum(pesoY)

    #Somatoria do pedo X
    somatorioPesoX = sum(pesoX)

    #Realiza combinação
    combinacaoX = combinacao(pesoX, coordenadasX)
    combinacaoY = combinacao(pesoY, coordenadasY)

    #Calcula a media dos pontos X
    for i in combinacaoX:

        aux = calculaMedia(i)

        mediaX.append(aux)

    #Calcula a media dos pontos Y
    for i in combinacaoY:

        aux = calculaMedia(i)

        mediaY.append(aux)


    somatorioX =  sum(mediaX)
    somatorioY =  sum(mediaY)

    pontoX = (somatorioX / somatorioPesoX)  
    pontoY = (somatorioY / somatorioPesoY)

    pontoX = int(pontoX)
    pontoY = int(pontoY)
    
    #print(pontoX, pontoY)

    return pontoX, pontoY

In [6]:
def verifica_pixel(img, pontoY, pontoX):
            
    encontrou = 0
    
    try:

        if np.any(img[int(pontoY), int(pontoX)] == (0,0,0)):

            encontrou = 0
            #print("veri", encontrou)

        else:

            encontrou = 1
            #print("veri", encontrou)

        return encontrou
    
    except IndexError:
        
        encontrou = 1
        
        return encontrou

In [28]:
def localizaPonto(img, pontoY, pontoX):
    
    copiaX, copiaY = pontoX, pontoY
    
    largura = img.shape[0]
    altura  = img.shape[1]
        
    larguraCentro = largura / 2
    alturaCentro  = altura /  2
        
    qtdBusca = 1
    
    while(1):
        
        #print("x - y: ", pontoX, pontoY)
        
        coordenada = verifica_pixel(img, round(pontoY), round(pontoX))
                
        try:
    
            if(coordenada == 1):

                pontoY, pontoX = pontoY, pontoX
                
                return int(pontoY), int(pontoX), qtdBusca

            elif((pontoX > 2) and (pontoX < largura - 2)):
                                    
                if(pontoX >= larguraCentro):
                
                    pontoX = pontoX + 1
                    
                else:
                    
                    pontoX = pontoX - 1
                
            else:
                
                pontoY = pontoY + 1
                
                #return int(copiaY), int(copiaX), qtdBusca  
                    
            qtdBusca = qtdBusca + 1 
            
        except IndexError:
                        
            return int(copiaY), int(copiaX), qtdBusca

# IA

# 0. Setup Paths

In [8]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

import os

# 1. Create Label Map

In [9]:
labels = [{'name':'manga', 'id':1}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [10]:
labels

[{'name': 'manga', 'id': 1}]

# 2. Create TF records

In [4]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [ ]:
#!cd Tensorflow && git clone https://github.com/tensorflow/models

In [10]:
#if os.name=='nt':
  #  !pip install wget
   # import wget

In [ ]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [11]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [12]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

J  existe uma subpasta ou um arquivo Tensorflow\workspace\models\my_ssd_mobnet.


# 5. Update Config For Transfer Learning

In [13]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [14]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [15]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [16]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [17]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [18]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [19]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [14]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


# 7. Load Train Model From Checkpoint

In [20]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [21]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [22]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from scipy import spatial

In [23]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [24]:
category_index

{1: {'id': 1, 'name': 'manga'}}

In [36]:
idx = 1

media = []

while(idx <= 106):

    img = cv2.imread("Segundo_Teste/(" + str(idx) + ").jpg")
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    imagem_pedunculo = image_np.copy()
    
    detection_threshold = 0.7

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                skip_scores= True,
                skip_labels= True,
                agnostic_mode=False)
    
    scores = list(filter(lambda x: x> detection_threshold, 
    detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    media.append(scores)

    width = image_np_with_detections.shape[1]
    height = image_np_with_detections.shape[0]
    
    for idxb, box in enumerate(boxes):
            
        roi = box*[height, width, height, width]

        #Area de corte da imagem

        xt = roi[1]
        xb = roi[3]
        yt = roi[0]
        yb = roi[2]

        area = preveArea(xt, yt, xb, yb, 0.2, 0.5) #Função que preve a area

        x1, y1, x2, y2 = area[0], area[1], area[2], area[3] #Pega as coordenadas

        TopLeftX = x1
        TopLeftY = y2            

        corte_area_pedunculo = imagem_pedunculo[y2:y1, x1:x2] #Faz o corte na area prevista

        cv2.putText(image_np_with_detections, (str(idxb)), (x1, y2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255))

        cv2.imwrite("Primeiro_Teste_Imagens_Lindas_RESULTADOS_CORTES/" + str(idx) + ".jpg", corte_area_pedunculo) #Salva as imagens cortadas

        cv2.rectangle(image_np_with_detections, (x2, y2), (x1,y1), (0,0,255), 2) #Desenha um retangulo na area encontrada
        
        pedunculo_segmentado = segmentacaoHSV(corte_area_pedunculo) #Segmenta o pedunculo
        
        cv2.imwrite("Primeiro_Teste_Imagens_Lindas_RESULTADOS_SEGMENTACAO/" + str(idx) + ".jpg", pedunculo_segmentado)
        
        pontos_hue = maior_valor_H(idx, pedunculo_segmentado, 0.20) #Encontra os pontos HUE do pedunculo
    
        todas, x, y, vizualiza = pontos_hue[0], pontos_hue[1], pontos_hue[2], pontos_hue[3] #Pega as coordenadas
        
        cv2.imwrite("Primeiro_Teste_Imagens_Lindas_RESULTADOS_HUE/" + str(idx) + ".jpg", vizualiza)
        
        pontos = encontrarPontos(pedunculo_segmentado, todas, x, y) #Encontra o ponto de partida:

        pontoPartidaX, pontoPartidaY = pontos[1], pontos[0] #Pega a coordenada do ponto de partida (x, y)
        
        cv2.circle(vizualiza, (pontoPartidaY, pontoPartidaX), 5, (255, 255, 255), -1) #Ponto de partida

        cv2.imwrite("Primeiro_Teste_Imagens_Lindas_RESULTADOS_HUE_PONTO/"  + str(idx) + ".jpg", vizualiza)
        
        #print("ok")
        
        localizaPontoFinal = localizaPonto(pedunculo_segmentado, pontoPartidaX, pontoPartidaY)
        
        pontoBuscadoX, pontoBuscadoY, qtdPontos = localizaPontoFinal[1], localizaPontoFinal[0], localizaPontoFinal[2]

        pontoFinalX = abs(pontoBuscadoX)
        pontoFinalY = abs(pontoBuscadoY)

        #Encontra a coordenada final na imagem completa
        CDFX = TopLeftX + pontoFinalX
        CDFY = TopLeftY + pontoFinalY

        cv2.circle(image_np_with_detections, (CDFX, CDFY), 7, (0, 0, 255), -1)
        
        cv2.imwrite("Segundo_Teste_Resultados/" + str(idx) + ".jpg", image_np_with_detections)

        print("Imagem: {} - Buscas: {}".format(idx, qtdPontos))

        idx = idx + 1

Imagem: 1 - Buscas: 1
Imagem: 2 - Buscas: 1
Imagem: 3 - Buscas: 1
Imagem: 4 - Buscas: 1
Imagem: 5 - Buscas: 1
Imagem: 6 - Buscas: 1
Imagem: 7 - Buscas: 1
Imagem: 8 - Buscas: 1
Imagem: 9 - Buscas: 1
Imagem: 10 - Buscas: 1
Imagem: 11 - Buscas: 1
Imagem: 12 - Buscas: 1
Imagem: 13 - Buscas: 1
Imagem: 14 - Buscas: 1
Imagem: 15 - Buscas: 1
Imagem: 16 - Buscas: 1
Imagem: 17 - Buscas: 19
Imagem: 18 - Buscas: 1
Imagem: 19 - Buscas: 1
Imagem: 20 - Buscas: 1
Imagem: 21 - Buscas: 1
Imagem: 22 - Buscas: 1
Imagem: 23 - Buscas: 50
Imagem: 24 - Buscas: 1
Imagem: 25 - Buscas: 1
Imagem: 26 - Buscas: 1
Imagem: 27 - Buscas: 24
Imagem: 28 - Buscas: 1
Imagem: 29 - Buscas: 1
Imagem: 30 - Buscas: 1
Imagem: 31 - Buscas: 1
Imagem: 32 - Buscas: 1
Imagem: 33 - Buscas: 1
Imagem: 34 - Buscas: 20
Imagem: 35 - Buscas: 1
Imagem: 36 - Buscas: 1
Imagem: 37 - Buscas: 1
Imagem: 38 - Buscas: 1
Imagem: 39 - Buscas: 1
Imagem: 40 - Buscas: 1
Imagem: 41 - Buscas: 1
Imagem: 42 - Buscas: 327
Imagem: 43 - Buscas: 284
Imagem: 44 -